In [1]:
import psutil

def get_free_memory():
    memory = psutil.virtual_memory()
    return memory.available / (1024.0 ** 3)  # Convert bytes to gigabytes

print(f"Free CPU Memory: {get_free_memory():.2f} GB")


Free CPU Memory: 334.84 GB


In [2]:
import json
with open("/home/toibazd/Most_frequent_IPs.json", "r") as f:
    ips = json.load(f)

sorted_dict = sorted(ips.items(), key=lambda x: x[1], reverse=True)
most_frequent_ips = [item[0] for item in sorted_dict[:100]]


In [3]:
most_frequent_ips

['IPR004090',
 'IPR011701',
 'IPR002514',
 'IPR003719',
 'IPR002155',
 'IPR005750',
 'IPR001001',
 'IPR004604',
 'IPR011603',
 'IPR005252',
 'IPR010228',
 'IPR004733',
 'IPR004469',
 'IPR001645',
 'IPR004805',
 'IPR010949',
 'IPR011276',
 'IPR003753',
 'IPR001381',
 'IPR001241',
 'IPR000565',
 'IPR002298',
 'IPR004563',
 'IPR000277',
 'IPR006235',
 'IPR002150',
 'IPR003841',
 'IPR011530',
 'IPR001207',
 'IPR000015',
 'IPR001036',
 'IPR040423',
 'IPR001898',
 'IPR003474',
 'IPR006043',
 'IPR002692',
 'IPR015712',
 'IPR000212',
 'IPR010243',
 'IPR045018',
 'IPR007312',
 'IPR006710',
 'IPR004586',
 'IPR005667',
 'IPR001278',
 'IPR001360',
 'IPR026856',
 'IPR003675',
 'IPR002903',
 'IPR006419',
 'IPR003567',
 'IPR024922',
 'IPR004504',
 'IPR002745',
 'IPR003477',
 'IPR010096',
 'IPR001734',
 'IPR003801',
 'IPR022838',
 'IPR006276',
 'IPR002528',
 'IPR004396',
 'IPR005814',
 'IPR045187',
 'IPR006127',
 'IPR047640',
 'IPR039672',
 'IPR022893',
 'IPR000380',
 'IPR001233',
 'IPR002908',
 'IPR0

In [4]:
import torch
torch.backends.cuda.matmul.allow_tf32 = True
from datasets import Dataset
import os
import numpy as np

In [7]:
from collections import defaultdict
from sklearn.preprocessing import MultiLabelBinarizer
import csv
from tqdm.auto import tqdm


data_dict = defaultdict(list)
enc = MultiLabelBinarizer()

with open("/home/toibazd/Prot2IP_GO_filtered_MF.tsv", "r") as tsvfile:
    reader = csv.reader(tsvfile, delimiter = "\t")
    for row in tqdm(reader):
        key = row[0].split("prot_")[1].split(".")[0]
        iprs = eval(row[1])
        
        for ip in iprs:
            if ip in most_frequent_ips:
                data_dict[key].append(ip)

one_hot_encoded = enc.fit_transform(data_dict.values())
one_hot_encoded_dict = {key: value for key, value in zip(data_dict.keys(), one_hot_encoded)}


0it [00:00, ?it/s]

In [20]:
print(one_hot_encoded.shape)

IP_number = np.sum(one_hot_encoded, axis = 1)
count_gt_2 = np.sum(IP_number >= 1)

print(count_gt_2)

(21791, 100)
21791


In [21]:
print(len(one_hot_encoded_dict.keys()))

21791


In [22]:
import random
from concurrent.futures import ThreadPoolExecutor

directory = '/ibex/user/toibazd/InterPro_annotated_genomes/'
one_hot_encoded_sentences = {}

sentence_length = 40
sentences_per_IP = 200

# Set random seed for reproducibility
random.seed(42)

# Randomly choose 1000 files with seed 42
selected_files = os.listdir(directory)


# Define a function to process a file
def process_file(filename, IP):
    sentences = []

    filepath = os.path.join(directory, filename)

    with open(filepath, 'r') as file:
        content = file.read()
        words = content.strip().split()

        # Check if the key is in the file
        for i in range(19, len(words)-20):
            # Shuffle the indices of the words containing the key
            if IP in data_dict[words[i]]:
                if len(words) - i >= 21:
                    sentence = " ".join(words[i - 19:i + sentence_length - 19])
                    sentences.append(sentence)
    return sentences



for IP in tqdm(most_frequent_ips):
    one_hot_encoded_sentences[IP] = []
    sentences_count = 0

    # Use ThreadPoolExecutor for concurrent processing
    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(process_file, filename, IP) for filename in selected_files]
        for future in futures:
            sentences = future.result()
            one_hot_encoded_sentences[IP].extend(sentences)
            sentences_count += len(sentences)
            if sentences_count >= sentences_per_IP:
                break

  0%|          | 0/100 [00:00<?, ?it/s]

In [23]:
import json

with open('BERT_DNN_senteces.json', 'w') as f:
    json.dump(one_hot_encoded_sentences, f)

In [24]:
directory = '/ibex/user/toibazd/InterPro_annotated_genomes_for_testing/'
one_hot_encoded_sentences_2 = {}

sentence_length = 40
sentence_per_IP = 50
selected_files = os.listdir(directory)


# Iterate over keys
for IP in tqdm(most_frequent_ips):
    one_hot_encoded_sentences_2[IP] = []
    sentences_count=0
    
    # Iterate over selected files
    for filename in selected_files:
        filepath = os.path.join(directory, filename)

        with open(filepath, 'r') as file:
            content = file.read()
            words = content.strip().split()

            # Check if the key is in the file
            for i in range(19, len(words)-20):
                # Shuffle the indices of the words containing the key
                if IP in data_dict[words[i]]:
                    if len(words) - i >= 21:
                        sentence = " ".join(words[i - 19:i + sentence_length - 19])
                        one_hot_encoded_sentences_2[IP].append(sentence)
                        sentences_count += 1
                        if sentences_count>=sentence_per_IP:
                            break
        if sentences_count>=sentence_per_IP:
            break
    print(sentences_count)
            


  0%|          | 0/100 [00:00<?, ?it/s]

50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50


In [26]:
import json

with open('BERT_DNN_senteces_test.json', 'w') as f:
    json.dump(one_hot_encoded_sentences_2, f)